<a href="https://colab.research.google.com/github/daniyal1d/ChatBot-with-LangGraph/blob/main/ChatBot_with_LangGraph22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain langgraph google-api-python-client langchain[google-genai]

In [2]:
import os
from getpass import getpass

def inject_env_variable(key):
    if key not in os.environ:
        os.environ[key] = getpass(f"🔐 Enter your {key}: ")

inject_env_variable("GOOGLE_API_KEY")     # From Google Cloud Console
inject_env_variable("GOOGLE_CSE_ID")

🔐 Enter your GOOGLE_API_KEY: ··········
🔐 Enter your GOOGLE_CSE_ID: ··········


In [3]:
from googleapiclient.discovery import build
from langchain.tools import Tool

def run_google_lookup(prompt: str, max_hits: int = 3):
    service = build("customsearch", "v1", developerKey=os.environ["GOOGLE_API_KEY"])
    response = service.cse().list(q=prompt, cx=os.environ["GOOGLE_CSE_ID"], num=max_hits).execute()
    links = response.get("items", [])
    return "\n".join([f"{i+1}. {item['title']} - {item['link']}" for i, item in enumerate(links)])

search_tool = Tool(
    name="GoogleWebSearch",
    func=run_google_lookup,
    description="Web lookup using Google's Custom Search API."
)

toolset = [search_tool]

In [4]:
!pip install -U langchain langchain_core

In [5]:
!pip uninstall -y langchain langchain_core
!pip install -U langchain langchain_core langchain[google-genai]

Found existing installation: langchain 0.3.25
Uninstalling langchain-0.3.25:
  Successfully uninstalled langchain-0.3.25
Found existing installation: langchain-core 0.3.59
Uninstalling langchain-core-0.3.59:
  Successfully uninstalled langchain-core-0.3.59
  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.59-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain-0.3.25-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.3.59-py3-none-any.whl (437 kB)


In [6]:
!pip install --upgrade langchain_core
!pip install --upgrade langchain
from langchain.chat_models import init_chat_model
llm_core = init_chat_model("google_genai:gemini-2.0-flash")
llm_with_search = llm_core.bind_tools(toolset)

In [7]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class DialogueState(TypedDict):
    messages: Annotated[list, add_messages]

graph_flow = StateGraph(DialogueState)

def core_agent(state: DialogueState):
    return {"messages": [llm_with_search.invoke(state["messages"])]}

graph_flow.add_node("core", core_agent)


In [8]:
import json
from langchain_core.messages import ToolMessage

class ToolExecutor:
    def __init__(self, tools: list):
        self.tool_dict = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        last_msg = inputs.get("messages", [])[-1]
        results = []
        for tool_call in last_msg.tool_calls:
            outcome = self.tool_dict[tool_call["name"]].invoke(tool_call["args"])
            results.append(
                ToolMessage(
                    content=json.dumps(outcome),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": results}

graph_flow.add_node("tools", ToolExecutor(tools=toolset))

In [9]:
def tool_router(state: DialogueState):
    last_msg = state["messages"][-1]
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "tools"
    return END

graph_flow.add_conditional_edges("core", tool_router, {"tools": "tools", END: END})
graph_flow.add_edge("tools", "core")
graph_flow.add_edge(START, "core")

conversation_graph = graph_flow.compile()


In [10]:
def launch_chat(prompt_text: str):
    for update in conversation_graph.stream({"messages": [{"role": "user", "content": prompt_text}]}):
        for packet in update.values():
            print("🤖:", packet["messages"][-1].content)

In [11]:
print("✨ Gemini Search Assistant (type 'quit' to exit)\n")
while True:
    try:
        user_query = input("👤: ")
        if user_query.strip().lower() in ["exit", "quit", "q"]:
            print("🛑 Session Ended.")
            break
        launch_chat(user_query)
    except Exception as e:
        print("⚠️ Error occurred:", str(e))
        break

✨ Gemini Search Assistant (type 'quit' to exit)

👤: hello
🤖: Hello! How can I help you today?
👤: exit
🛑 Session Ended.


In [12]:
# Upload your `credentials.json` file
from google.colab import files
uploaded = files.upload()

Saving sacred-alliance-459709-p7-fa2d4b51835d.json to sacred-alliance-459709-p7-fa2d4b51835d (1).json


In [13]:
!pip install --upgrade google-auth-oauthlib
!pip install --upgrade google-auth-oauthlib google-auth google-api-python-client

In [14]:
# Authenticate with Google Calendar using OAuth
import os
import datetime
import pickle
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/calendar']

def get_calendar_service():
    creds = None
    if os.path.exists('token.pkl'):
        with open('token.pkl', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Replace 'credentials.json' with the correct filename
            flow = InstalledAppFlow.from_client_secrets_file("sacred-alliance-459709-p7-fa2d4b51835d.json", SCOPES) # This line was modified
            creds = flow.run_local_server(port=0)
        with open('token.pkl', 'wb') as token:
            pickle.dump(creds, token)
    service = build("calendar", "v3", credentials=creds)
    return service

calendar_service = get_calendar_service()

ValueError: Client secrets must be for a web or installed app.

In [15]:
import os
import openai
from langchain.agents import Tool
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_types import AgentType
from langchain.chains.llm_math.base import LLMMathChain

In [16]:
import os
import openai
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.agents import Tool
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

In [17]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import datetime

# Path to your service account key file
SERVICE_ACCOUNT_FILE = '/content/sacred-alliance-459709-p7-fa2d4b51835d.json'

# Scopes required for Google Calendar
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

# Authenticate
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Build the service
service = build('calendar', 'v3', credentials=credentials)

# Get upcoming events
now = datetime.datetime.utcnow().isoformat() + 'Z'
events_result = service.events().list(
    calendarId='primary', timeMin=now,
    maxResults=5, singleEvents=True,
    orderBy='startTime').execute()
events = events_result.get('items', [])

for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])

In [18]:
def get_upcoming_events():
    from google.oauth2 import service_account
    from googleapiclient.discovery import build
    import datetime

    SERVICE_ACCOUNT_FILE = 'sacred-alliance-459709-p7-fa2d4b51835d.json'
    SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('calendar', 'v3', credentials=credentials)

    now = datetime.datetime.utcnow().isoformat() + 'Z'
    events_result = service.events().list(
        calendarId='primary', timeMin=now,
        maxResults=5, singleEvents=True,
        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        return "No upcoming events found."

    response = ""
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        summary = event.get('summary', 'No title')
        response += f"{start}: {summary}\n"
    return response.strip()

In [19]:
from langchain.agents import Tool

calendar_tool = Tool(
    name="Google Calendar",
    func=lambda _: get_upcoming_events(),
    description="Use this to check upcoming calendar events."
)

In [20]:
!pip install -U langchain langchain-community google-auth google-api-python-client

In [22]:
!pip install -U langchain_google_genai  # Make sure package is up-to-date
from langchain_google_genai import ChatGoogleGenerativeAI  # Import the class

llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

  Using cached langchain_google_genai-2.1.4-py3-none-any.whl.metadata (5.2 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached langchain_google_genai-2.1.4-py3-none-any.whl (44 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
  Attempting uninstall: langchain_google_genai
    Found existing installation: langchain-google-genai 2.0.10
    Uninstalling langchain-google-genai-2.0.10:
      Successfully uninstalled langchain-google-genai-2.0.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativel

In [23]:
from google.generativeai import list_models
import google.generativeai as genai

genai.configure(api_key="API")  # Replace with your key

models = list_models()
for m in models:
    print(m.name)


models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-p

In [25]:
!pip install -U langchain langchain-google-genai google-generativeai

import os
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI

# Set your actual Gemini API key
os.environ["GOOGLE_API_KEY"] = "Api"

llm = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp", temperature=0.3)

def dummy_weather_tool(query: str) -> str:
    return "It's 28°C and sunny today."

weather_tool = Tool(
    name="WeatherTool",
    func=dummy_weather_tool,
    description="Useful for getting current weather info."
)

agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent.run("What's the weather like today?")
print("\nFinal Response:\n", response)


  Using cached langchain_google_genai-2.1.4-py3-none-any.whl.metadata (5.2 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
INFO: This is taking longer than usual. You might need to pro

In [26]:
!pip install -U langchain langchain-google-genai google-auth google-api-python-client

import os
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI

# ✅ Set your Gemini API key here
os.environ["GOOGLE_API_KEY"] = "APi"

# ✅ Initialize Gemini Pro
llm = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp", temperature=0.3)

# ✅ Dummy calendar event tool functions
def calendar_event_tool(query: str) -> str:
    query_lower = query.lower()

    if "meeting" in query_lower and "today" in query_lower:
        return "You have 2 meetings today: one at 10 AM and one at 3 PM."
    elif "schedule" in query_lower:
        return "Event scheduled: Meeting at 3 PM has been added to your calendar."
    elif "tomorrow" in query_lower:
        return "No meetings scheduled for tomorrow."
    else:
        return "I couldn't find any events matching your query."

# ✅ Register tool
calendar_tool = Tool(
    name="CalendarTool",
    func=calendar_event_tool,
    description="Useful for scheduling or checking meetings and calendar events."
)

# ✅ Create the agent
agent = initialize_agent(
    tools=[calendar_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# ✅ Sample test
response = agent.run("Do I have any meetings today?")
print("\nFinal Response:\n", response)


  Using cached langchain_google_genai-2.1.4-py3-none-any.whl.metadata (5.2 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached langchain_google_genai-2.1.4-py3-none-any.whl (44 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
  Attempting uninstall: langchain-google-genai
    Found existing installation: langchain-google-genai 2.0.10
    Uninstalling langchain-google-genai-2.0.10:
      Successfully uninstalled langchain-google-genai-2.0.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativel



> Entering new AgentExecutor chain...
Thought: I need to check the calendar for any meetings scheduled for today. The CalendarTool can help with this.
Action: CalendarTool
Action Input: "What are my meetings scheduled for today?"
Observation: You have 2 meetings today: one at 10 AM and one at 3 PM.
Thought:I now know the final answer
Final Answer: You have 2 meetings today: one at 10 AM and one at 3 PM.

> Finished chain.

Final Response:
 You have 2 meetings today: one at 10 AM and one at 3 PM.


In [28]:
!pip install -U langchain langchain-google-genai google-auth google-api-python-client

import os
from datetime import datetime
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain_google_genai import ChatGoogleGenerativeAI

# 🔐 Set up your Gemini API key
os.environ["GOOGLE_API_KEY"] = "api"  # Replace with your actual key

# 🧠 Initialize the Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp", temperature=0.2)

# 📅 Simulated event memory (can be replaced with database or API later)
calendar_events = {
    "2025-05-13": ["Team sync at 10:00 AM", "Client call at 2:00 PM"],
    "2025-05-14": [],
}

# 🔧 Custom scheduling logic
def smart_scheduler(query: str) -> str:
    today = datetime.today().strftime('%Y-%m-%d')
    query_lower = query.lower()

    if "today" in query_lower and "meeting" in query_lower:
        events = calendar_events.get(today, [])
        return f"Today's events: {', '.join(events)}" if events else "You have no meetings today."

    elif "add" in query_lower or "schedule" in query_lower:
        # Simulate adding an event
        new_event = "Strategy meeting at 4:00 PM"
        calendar_events[today].append(new_event)
        return f"Scheduled: {new_event}"

    elif "tomorrow" in query_lower:
        tomorrow = (datetime.today().date().fromisoformat(today) + timedelta(days=1)).isoformat()
        events = calendar_events.get(tomorrow, [])
        return f"Tomorrow's events: {', '.join(events)}" if events else "Your schedule is free tomorrow."

    else:
        return "I wasn't able to process your calendar request."

# 🛠️ Register the tool
scheduler_tool = Tool(
    name="SmartSchedulerTool",
    func=smart_scheduler,
    description="Checks or creates calendar events based on natural language requests."
)

# 🤖 Initialize LangChain agent
agent = initialize_agent(
    tools=[scheduler_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 🧪 Run test query
response = agent.run("Can you schedule a new meeting for today at 4 PM?")
print("\nAgent's Response:\n", response)




> Entering new AgentExecutor chain...
Action: SmartSchedulerTool
Action Input: Schedule a new meeting for today at 4 PM.
Observation: Today's events: Team sync at 10:00 AM, Client call at 2:00 PM
Thought:Thought: The user asked me to schedule a new meeting for today at 4 PM. I used the SmartSchedulerTool with the request "Schedule a new meeting for today at 4 PM." The tool's observation was "Today's events: Team sync at 10:00 AM, Client call at 2:00 PM".

This observation lists the current events for today. Crucially, the requested 4 PM meeting is *not* in this list. If the tool had successfully scheduled the meeting and then reported the day's events, the new meeting should have been included. Since it's not, I must conclude that the meeting was not scheduled by this tool action, or at least the tool did not confirm its creation in the list of events.

However, the list of existing events (10:00 AM and 2:00 PM) indicates that 4:00 PM is indeed available.

So, to answer the question 